# Práctica 4 (opcional). Relación entre entidades en Wikidata.

# Grupo 05

* **Boris Carballa Corredoira**
* **Juan Carlos Villanueva Quirós**
* **Francisco Javier Blázquez Martínez**

Uno de los usos más interesantes de las redes semánticas es calcular la relación entre dos entidades recorriendo las aristas del grafo hasta encontrar una entidad común. Por ejemplo, utilizando sólo la relación _subclass of_ podemos calcular la relación entre las entidades [piano (Q5994)](https://www.wikidata.org/wiki/Q5994) y [electronic keyboard (Q1343007)](https://www.wikidata.org/wiki/Q1343007) que es la siguiente:

    piano (Q5994) -> subclass of (P279) -> keyboard instrument (Q52954)
    electronic keyboard (Q1343007) -> subclass of (P279) -> keyboard instrument (Q52954)
    
En este caso las dos entidades representan instrumentos que son de teclado. Además, podemos calcular una distancia semántica entre ellas sumando las longitudes de los caminos que llevan desde cada una de ellas a la entidad común. En este caso sólo hemos tenido que recorrer una arista desde cada una de ellas para llegar a _keyboard instrument_ así que podemos decir que se encuentran a distancia 1+1=2 y que la entidad común está a distancia 1 de cada una de ellas.

El algoritmo que calcula la entidad común más cercana lo hemos visto en clase. Se basa en crear una onda a partir de cada una de las entidades que estamos comparando y expandirlas alternativamente hasta que intersecan.

## Parte 1

Implementa una función que, dados dos identificadores de ítems y una distancia máxima, devuelva los ítems comunes que están como máximo a esa distancia de los dos ítems iniciales. 

Como Wikidata es muy grande vamos a limitarnos a buscar entidades comunes a distancia máxima 3 usando sólo el siguiente subconjunto de propiedades:

    instance of (P31), subclass of (P279), has part (P527), 
    instrument (P1303), genre (P136)
    
Si no hay entidades comunes a esa distancia diremos que los ítems no están relacionados. 

Calcula las relaciones entre las siguientes entidades dos a dos y explica en lenguaje natural el tipo de relación encontrada:

    piano (Q5994), electronic keyboard (Q1343007), 
    String synthesizer (Q2355465), Hans Zimmer (Q76364),
    Pirates of the Caribbean: The Curse of the Black Pearl (Q46717), 
    The Lion King (Q36479), Toy Story (Q171048), Iron Man (Q192724)
    
Consejos:

- Obtener la información de una entidad Wikidata con _get_entity_dict_from_api_ es una operación lenta. Te recomendamos que crees una cache de entidades (map entidad_id -> entidad) que evite pedir una y otra vez las mismas entidades a Wikidata.
- Te recomendamos crear una función _get_claims(item_id)_ que devuelva las relaciones que salen a partir de una entidad en formato (id_propiedad, id_valor). Sólo debes considerar los _truthy_claims_ que tengan como valor un item de Wikidata (entidades que empiezan por ‘Q’). Recuerda que también hemos limitado las propiedades que debes considerar.
- Es importante tener en cuenta que, cuando expandimos una onda, la intersección se puede producir con cualquiera de las entidades de la otra onda (y no sólo con las de la frontera).

## Parte 2

Implementa una versión avanzada del algoritmo visto en clase que no sólo devuelva las entidades comunes sino también los caminos que conectan cada una de las entidades originales con esas entidades comunes. En este caso, la función sólo debe devolver los caminos de longitud mínima. Si no hay caminos de longitud menor o igual a 5 diremos que las entidades no están relacionadas.

Vuelve a comparar las entidades del apartado anterior dos a dos mostrando los caminos de longitud mínima. Sólo es necesario considerar las propiedades indicadas previamente.

Consejos:

- La intersección de las dos ondas puede contener varias entidades. Debes considerar todas ellas a la hora de calcular las posibles soluciones. Además, cada una de las entidades comunes puede dar lugar a varias soluciones si hay distintos caminos que conectan las entidades iniciales con esa entidad común.
- De todas las soluciones posibles sólo debes devolver las de longitud mínima, entendiendo que la longitud de la solución es la suma de las longitudes de los caminos desde las entidades iniciales hasta la entidad común.
- Hay muchas formas de implementar este algoritmo. Una de ellas consiste en almacenar todos los caminos de cada una de las ondas e ir expandiéndolos alternativamente hasta que las ondas intersequen. Para calcular la intersección deberás almacenar también los ítems de cada una de las ondas.
- Ten en cuenta que para calcular todas las posibles soluciones de longitud <= n debes expandir cada una de las ondas n veces (puede que la relación vaya del primer ítem hasta el segundo).

__Nota: se puede obtener la máxima calificación entregando sólo la parte 2 si se explica en lenguaje natural las relaciones encontradas.__

In [1]:
# Comando de instalación
# %pip install qwikidata

In [2]:
from qwikidata.entity import WikidataItem, WikidataProperty, WikidataLexeme 
from qwikidata.linked_data_interface import get_entity_dict_from_api

In [3]:
propsDescrip = {
    'P31' : "instance of",
    'P279' : "subclass of",
    'P527' : "has part",
    'P1303' : "instrument",
    'P136' : "genre"
}
props = propsDescrip.keys()

In [4]:
# mapa idEntidad->Entidad
# La entidad resultante necesita ser parseada por alguno de los constructores de Wikidata
mapEntity = {}
def getEntity(idEnt):
    if idEnt not in mapEntity:
        mapEntity[idEnt] = get_entity_dict_from_api(idEnt)
    return mapEntity[idEnt]

In [5]:
# Los caminos son listas de tuplas, donde el primer elemento de la tupla es la propiedad
# usada para conseguir la entidad que es el segundo elemento de la tupla
# Esta función devuelve las posibles continuaciones directas de ascendencia en el grafo
# usando las propiedades props para la entidad con código idEnt
def getClaims(idEnt):
    result = []
    item = WikidataItem(getEntity(idEnt))
    for p in props:
        claimGroup = item.get_truthy_claim_group(p)
        for claim in claimGroup:
            if claim.mainsnak.datavalue is None:
                continue
            idClaim = claim.mainsnak.datavalue.value['id']
            if idClaim[0]=='Q':
                result.append((p,idClaim))
    return result

In [6]:
# Recibe una lista con los caminos de mayor longitud hasta el momento: n
# Devuelve los caminos con longitud n+1
def expandirOnda(listaCam):
    result = []
    for cam in listaCam:
        continuaciones = getClaims(cam[-1][1])
        for cont in continuaciones:
            extensionCam = list(cam)
            extensionCam.append(cont)
            result.append(extensionCam)
    return result

In [7]:
# Girar es simplemente para que siempre queden los de Q1 en el lado izq de la
# tupla y los de Q2 en el lado derecho
# Calcula si algún final de algún camino de la lista de caminos nuevaListaCamQj
# coincide con el final de algún camino de alguna longitud de listaLongQi
def intersec(listaLongQi, nuevaListaCamQj, girar):
    result = [] # Lista de tuplas, donde cada elemento de la tupla es un camino
    for camQj in nuevaListaCamQj:
        for listaCamQi in listaLongQi:
            for camQi in listaCamQi:
                if camQi[-1][1]==camQj[-1][1]:
                    result.append((camQj,camQi) if girar else (camQi,camQj))
    return result

In [8]:
# Muestra un camino entre dos nodos mostrando el camino de ambos hasta el nodo común,
# mostrando el nombre de las propiedades y entidades
# Recibe una lista de tuplas donde cada elemento de la tupla es un camino al nodo común
# NOTA: todos los caminos empiezan con (0,Qi) que es el nodo en el que empezamos
def show(sols):
    print("Hay "+str(len(sols))+(" solucion" if len(sols)==1 else " soluciones")+
          " de longitud "+str(len(sols[0][0])-1 + len(sols[0][1])-1))
    for i in range(len(sols)): #Cada solución
        print('')
        print("Solución "+str(i+1)+":")
        t = sols[i]
        for j in range(2): # Los dos caminos
            q = t[j][0][1]
            qLabel = WikidataItem(getEntity(q)).get_label()
            print("Camino desde "+q+":")
            print(qLabel+" ("+q+")",end='')
            for k in range(1,len(t[j])):
                #Propiedad
                p = t[j][k][0]
                print(" -> "+propsDescrip[p]+" ("+p+") -> ",end='')
                q = t[j][k][1]
                qLabel = WikidataItem(getEntity(q)).get_label()
                print(qLabel+" ("+q+")",end='')
            print()
    print("----------")

In [9]:
# Cuando propagamos la onda un nivel más y comprobamos intersecciones de los finales
# las intersecciones comprobadas son las de finales de estos niveles:
# (sabiendo que Q1 y Q2 son distintos)
# Q1: 1 -> 0 de Q2
# Q2: 1 -> 0,1 de Q1
# Q1: 2 -> 0,1 de Q2
# Q2: 2 -> 0,1,2 de Q1
# Q1: 3 -> 0,1,2 de Q2
# Q2: 3 -> 0,1,2,3 de Q1
def caminos(Q1,Q2):
    if Q1==Q2:
        print("¡Son iguales!")
        return
    # cam1 es una lista de caminos clasificados por longitudes
    # cam1[a] es una lista de caminos de longitud a
    # cam1[a][b] es un camino (b-ésimo, pero no importa el orden) de longitud a
    # cam1[a][b][c] es la tupla c-ésima del camino anterior
    
    # Podríamos pensar en llevar siempre solo los caminos de la mayor longitud y comprobar
    # su intersección iterando por sus entidades en vez de solo por la última del camino
    # Esto nos daría muchos caminos repetidos pues habrá muchos caminos con una primera parte
    # idéntica. Esta versión aumenta la memoria usada pero disminuye el tiempo de comprobación.
    cam1 = [[[(0,Q1)]]]
    cam2 = [[[(0,Q2)]]]
    cam1a2 = []
    for long in range(5): #Se expandirá hasta los de longitud 5 (incluido)
        # Expandimos Q1
        nuevaListaCamQ1 = expandirOnda(cam1[long]) # tendrá longitud long+1
        # Comprobamos las coincidencias con los finales de los caminos de Q2
        cam1a2.extend(intersec(cam2,nuevaListaCamQ1,True))
        # Añadimos los nuevos caminos a la lista de caminos de Q1
        cam1.append(nuevaListaCamQ1)
        
        #Expandimos Q2
        nuevaListaCamQ2 = expandirOnda(cam2[long]) # tendrá longitud long+1
        # Comprobamos las coincidencias con los finales de los caminos de Q1
        cam1a2.extend(intersec(cam1,nuevaListaCamQ2,False))
        # Añadimos los nuevos caminos a la lista de caminos de Q2
        cam2.append(nuevaListaCamQ2)
    
    if len(cam1a2)==0:
        print("No están conectados")
        print("----------")
    else :
        longMin = min(map(lambda t: len(t[0])-1 + len(t[1])-1, cam1a2))
        if longMin>5 :
            print("No están conectados")
            print("----------")
        else:
            sol = list(filter(lambda t: (len(t[0])-1 + len(t[1])-1)==longMin, cam1a2))
            show(sol)

In [10]:
ent = ['Q5994','Q1343007','Q2355465','Q76364','Q46717','Q36479','Q171048','Q192724']
a = 0
for i in range(len(ent)):
    q1 = ent[i]
    for j in range(i+1,len(ent)):
        q2 = ent[j]
        print('('+str(a+1)+") "+q1 + " - " + q2)
        caminos(q1,q2)
        a += 1

(1) Q5994 - Q1343007
Hay 1 solucion de longitud 2

Solución 1:
Camino desde Q5994:
piano (Q5994) -> subclass of (P279) -> keyboard instrument (Q52954)
Camino desde Q1343007:
electronic keyboard (Q1343007) -> subclass of (P279) -> keyboard instrument (Q52954)
----------
(2) Q5994 - Q2355465
Hay 3 soluciones de longitud 4

Solución 1:
Camino desde Q5994:
piano (Q5994) -> subclass of (P279) -> keyboard instrument (Q52954) -> instance of (P31) -> class of instruments (Q1254773)
Camino desde Q2355465:
String synthesizer (Q2355465) -> instance of (P31) -> electronic musical instrument (Q1327500) -> instance of (P31) -> class of instruments (Q1254773)

Solución 2:
Camino desde Q5994:
piano (Q5994) -> subclass of (P279) -> true board zithers with resonator box (Q4951628) -> instance of (P31) -> class of instruments (Q1254773)
Camino desde Q2355465:
String synthesizer (Q2355465) -> instance of (P31) -> electronic musical instrument (Q1327500) -> instance of (P31) -> class of instruments (Q12547

Explicación en lenguaje natural de las relaciones encontradas:

1)\
El piano y el teclado electrónico son tipos de instrumentos con teclado

2)\
2.1)\
El piano y el sintetizador de cuerda son instrumentos, pues el piano es un tipo de instrumento con teclado y el sintetizador de cuerda es un instrumento musical electrónico.\
2.2)\
El piano y el sintetizador de cuerda son instrumentos, el sintetizador de cuerda lo es por la misma razón pero el piano es instrumento porque es un tipo de cítara con caja de resonancia.\
2.3)\
El piano y el sintetizador de cuerda son instrumentos musicales, pues el piano lo es y el sintetizador de cuerda es un instrumento musical electrónico, que es un tipo de electrófono, que es un tipo de instrumento musical.

3)\
Hans Zimmer toca el piano

4) 5) 6) 7)\
No están conectados

8)\
Un teclado electrónico es un tipo de instrumento musical electrónico y un syntetizador de cuerda es un instrumento musical electrónico.

9)\
El teclado electrónico es un tipo de intrumento de cuerda y Hans Zimmer tocaba instrumentos de cuerda.

10) 11) 12) 13)\
No están conectados

14)\
Un sintetizador de cuerda es un instrumento musical electrónico y Hans Zimmer tocaba el sintetizador, que es un tipo de instrumento musical electrónico.

15) 16) 17) 18) 19) 20) 21) 22)\
No están conectados

23)\
23.1)\
Piratas del Caribe y El Rey León son películas, siendo El Rey León una película de tipo animada\
23.2)\
Piratas del Caribe y El Rey León son películas, siendo El Rey León una película de género musical\
23.3)\
Piratas del Caribe y El Rey León son películas, siendo El Rey León una película de género infantil\
23.4)\
Piratas del Caribe y El Rey León son películas, siendo El Rey León una película de género dramático

24)\
Piratas del Caribe y Toy Story son películas de género fantástico

25)\
25.1)\
Piratas del Caribe e Iron Man son películas\
25.2)\
Piratas del Caribe e Iron Man son del género películas de acción\
25.3)\
Piratas del Caribe e Iron Man son del género de películas de aventuras

26)\
El Rey León y Toy Story son películas animadas, siendo Toy Story una película de animación de larga duración

27)\
27.1)\
El Rey León e Iron Man son películas, siendo El Rey León una película animada\
27.2)\
El Rey León e Iron Man son películas, siendo El Rey León una película de género musical\
27.3)\
El Rey León e Iron Man son películas, siendo El Rey León una película de género infantil\
27.4)\
El Rey León e Iron Man son películas, siendo El Rey León una película de género dramático

28)\
Toy Story e Iron Man son películas, siendo Toy Story una película del género película de amigos